# Product availability dashboard

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json
import datetime as dt
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from tqdm import tqdm
from viresclient import SwarmRequest

import panel as pn
import hvplot.xarray
import holoviews as hv
import geoviews as gv

hv.extension('bokeh')
pn.extension('ace')

In [ ]:
SERVER_URL = "https://vires.services/ows"

## Retrieving availability of collections from VirES

In [ ]:
def fetch_availability(force_update=False):
    if ("availability" not in pn.state.cache) or force_update:
        collections_dict = SwarmRequest().available_collections(details=False)
        # Choose only the main collections
        collections_dict = {
            c: collections_dict[c] for c in [
                "MAG", "MAG_HR", "EFI", "TEC", "FAC", "EEF", "IPD",
                "AEJ_LPL", "AEJ_LPS", "AEJ_PBL", "AEJ_PBS", "AOB_FAC"
            ]
        }
        request = SwarmRequest(SERVER_URL)
        availability = {}
        for group in tqdm(collections_dict.keys()):
            for collection in collections_dict[group]:
                df = request.available_times(collection)
                # Create integer index given by YearDOY 
                year = df["starttime"].dt.year
                doy = df["starttime"].dt.dayofyear
                index = np.array([f"{x}{y:03}" for x, y in zip(year, doy)]).astype(int)
                df["year+doy"] = index
    #             availability[collection].index = index
    #             availability[collection].index.name = "year+doy"
                # Remove unused bbox column
                df = df.drop(columns=["bbox"])
                # Add column containing just the collection (product) name
                df["Product"] = collection
                # Add column containing just the collection group
                df["Group"] = group
                # Append to dictionary
                availability[collection] = df
        # Reorganise dictionary into MultiIndex DataFrame
        availability = pd.concat(availability.values())
        availability.index = pd.MultiIndex.from_frame(
            availability[["Group", "Product", "year+doy"]]
        )
        availability = availability.drop(columns=["year+doy", "Product", "Group"])
        pn.state.cache["availability"] = availability
    else:
        availability = pn.state.cache["availability"]
    return availability

# availability = fetch_availability(force_update=True)

In [ ]:
# availability.head()
# # Selecting a particular collection
# availability.xs("SW_OPER_MAGA_LR_1B", level=1)
# # Selecting a particular year
# availability.xs(slice(2013000, 2014000), level=2)

## Creating a Holoviews object to display availability for a given year

In [ ]:
def segments_for_year(year=2020):
    """Set year to None to display all time"""
    availability = fetch_availability()
    # Slice out chosen year
    if year:
        selection = availability.xs(
            slice(int(f"{year}000"), int(f"{year+1}000")),
            level=2
        )
    else:
        selection = availability
    # Generate segments
    segs = hv.Segments(
        selection,
        ["starttime", "Product", "endtime", "Product"]
    ).opts(
        line_width=10, xlabel="Day",
        responsive=True, min_width=800, min_height=800, max_height=800,
        tools=["hover"], active_tools=[]
    )
    if year:
        segs = segs.opts(xlim=(dt.datetime(year, 1, 1), dt.datetime(year+1, 1, 1)))
    return segs

In [ ]:
# segments_for_year(2020)

## Repeating for model availability

In [ ]:
def fetch_availability_of_models(force_update=False):
    def _parsetime(t):
        return dt.datetime.fromisoformat(t[:-1])
    if ("availability_models" not in pn.state.cache) or force_update:
        request = SwarmRequest(SERVER_URL)
        model_info = request.get_model_info()
        d = {
            i: [] for i in ["Model name", "start", "end", "expression", "sources"]
        }
        for m in model_info.keys():
            d["Model name"].append(m)
            d["start"].append(_parsetime(model_info[m]["validity"]["start"]))
            d["end"].append(_parsetime(model_info[m]["validity"]["end"]))
            d["expression"].append(model_info[m]["expression"])
            d["sources"].append("; ".join(model_info[m]["sources"]))
        availability_models = pd.DataFrame.from_dict(d)
#         availability_models = availability_models.set_index("Model name")
        pn.state.cache["availability_models"] = availability_models
    else:
        availability_models = pn.state.cache["availability_models"]
    return availability_models

# availability_models = fetch_availability_of_models(force_update=True)
# availability_models.head()

In [ ]:
def segments_model_availability():
    model_availability = fetch_availability_of_models()
    # Generate segments
    segs = hv.Segments(
        model_availability,
        ["start", "Model name", "end", "Model name"]
    ).opts(
        line_width=10, xlabel="Time", xlim=(dt.datetime(2010, 1, 1), dt.datetime(2025, 1, 1)),
        responsive=True, min_width=800, min_height=800, max_height=800,
        tools=["hover"], active_tools=[]
    )
    return segs

# segments_model_availability()

## Repeating for AUX_OBS availability

In [ ]:
def fetch_availability_of_obs(force_update=False):
    if ("availability_obs" not in pn.state.cache) or force_update:
        request = SwarmRequest(SERVER_URL)
        available_aux_obs = {}
        available_aux_obs["H"] = request.available_observatories("SW_OPER_AUX_OBSH2_", details=True)
        available_aux_obs["M"] = request.available_observatories("SW_OPER_AUX_OBSM2_", details=True)
        available_aux_obs["S"] = request.available_observatories("SW_OPER_AUX_OBSS2_", details=True)
        for hms in tqdm("HMS"):
            for se in ("startTime", "endTime"):
                available_aux_obs[hms][se] = pd.to_datetime(
                    available_aux_obs[hms][se]
                )
            # compatibility with old vires
            available_aux_obs[hms] = available_aux_obs[hms].rename(
                columns={"IAGACode": "site"}
            )
        pn.state.cache["availability_obs"] = available_aux_obs
    else:
        available_aux_obs = pn.state.cache["availability_obs"]
    return available_aux_obs

# available_aux_obs = fetch_availability_of_obs(force_update=True)

In [ ]:
def segments_for_aux_obs(hms="H"):
    df = fetch_availability_of_obs()[hms]
#     df = df.sort_values(["endTime", "startTime"])
    
    segs = hv.Segments(
        df,
        ["startTime", "site", "endTime", "site"]
    ).opts(
        line_width=10, xlabel="Time", ylabel="IAGA Code",
        responsive=True, min_width=800, min_height=len(df)*12,
        tools=["hover"], active_tools=[],
#         xformatter='%d'
    )
    return segs

# segments_for_aux_obs("S")

## Assembling a Panel dashboard

In [ ]:
years = list(range(2013, 2022))
year_slider = pn.widgets.DiscreteSlider(name="Select year", options=years, value=years[-2])
hvplot_timeseries = pn.pane.HoloViews(segments_for_year(year_slider.value))
hvplot_models = pn.pane.HoloViews(segments_model_availability())
hvplot_obs_h = pn.pane.HoloViews(segments_for_aux_obs("H"))
hvplot_obs_m = pn.pane.HoloViews(segments_for_aux_obs("M"))
hvplot_obs_s = pn.pane.HoloViews(segments_for_aux_obs("S"))

def update_hvplot(event):
    year = event.obj.value
    hvplot_timeseries.object = segments_for_year(year)

year_slider.param.watch(update_hvplot, "value")
    
tab1 = pn.Column(
    hvplot_timeseries,
    year_slider
)
tab2 = pn.Column(
    hvplot_models
)
tab3 = pn.Tabs(
    ("Hour", hvplot_obs_h),
    ("Minute", hvplot_obs_m),
    ("Second", hvplot_obs_s)
)
dashboard = pn.Tabs(
    ("Time series products", tab1),
    ("Geomagnetic models", tab2),
    ("Geomagnetic ground observatories", tab3)
)
dashboard.servable(title="Data product availability on VirES")

```
docker run --rm -it -p 5006:5006 -v ~/code/Swarm_notebooks/dashboards:/home/jovyan \
registry.gitlab.eox.at/esa/vires_vre_ops/vre-swarm-notebook:0.7.5 \
bash -c \
    "conda upgrade --channel holoviz holoviews &&
    pip install --upgrade 'git+https://github.com/ESA-VirES/VirES-Python-Client@staging#egg=viresclient' &&
    viresclient set_token https://vires.services/ows &&
    panel serve 00_Product-Availability.ipynb --allow-websocket-origin=$(curl ifconfig.me/ip):5006"
```